# Cardio Data Analysis

## Contents

* About
    * Project Development
    * Problem Definition
    * Objective
* Data
    * Libraries
    * Importing
    * Variables
* Exploratory Data Analysis (EDA)
    * Plotting Objectives
    * Functions
    * Shape and Size
    * Types
    * Unique Values
    * Missing Values
    * Units Conversion
    * Continuous and Categorical Variables
        * Continuous Variables
            * Summary statistics
            * Probability Distribution
            * Making Sense of the (Continuous) Data
        * Categorical Variables
            * Bar Plots
            * Making Sense of the (Categorical) Data
    * Class Imbalance
* Feature Engineering
    * Units Conversion
    * Continuous Variables
        * Feature Scaling - Standardization (or Z-score Normalization)
        * Outliers Detection and Treatment
    * Categorical Variables
        * Label Encoding
* Feature Selection
    * Inferential Statistics and Hypothesis Testing
    * Feature Importance
    * Correlation Matrix Heatmap
* Model Training
* Model Evaluation
* Class Imbalance ?

## About

### Project Development
This project was developed locally with Visual Studio Code and GitHub version control.

Please check this project @ [GitHub page](https://caiocvelasco.github.io/) or @ [GitHub Repository - Cardio Data Analysis](https://github.com/caiocvelasco/health-data-analysis/blob/a4fafbcd8148a6d501f42a10ae9d313fc3b268e1/cardio-data-analysis-project.ipynb).

### Problem Definition

A client would like to understand some important patients' cardio-related descriptive statistics.

### Objective
Our goal is to calculate some descriptive statistics using Numpy, a package for scientific computing in Python.

## Data
Data was already available on a _csv_ format.

### Libraries

In [2]:
# !pip install seaborn pandas matplotlib numpy
import pandas as pd              # for data analysis
import numpy as np               # for scientific computing
import os                        # for file interactions in the user's operating system
import warnings                  # for dealing with warning messages if need be
warnings.filterwarnings('ignore')

import matplotlib.pyplot as plt  # for data visualization
# import matplotlib as mpl
import seaborn as sns            # for data visualization

### Importing

In [7]:
# Basic Settings
csv_folder_name = "health_dataset"  # please, change the folder name (where the CSV files are stored) if need be
notebook_location = "C:\\Users\\caiov\\OneDrive - UCLA IT Services\\Documentos\\DataScience\\Datasets" # set the location where this notebook is saved
csv_folder_path = notebook_location + "\\" + csv_folder_name  # set path for the CSV files
os.chdir(csv_folder_path)                                     # set location of CSV files

# Save cvs Data on a Pandas Dataframe
df = pd.read_csv("cardio_base.csv", sep = ",", skipinitialspace = True) #skip space after delimiter if need be

# Save a Copy of the Dataframe
data = df.copy()

# Dataset Manipulation
data.name = "Cardio Base Dataset" # rename the dataset 
cols = data.columns;              # create an index list with feature names

# Quick Overview of a Sample from the Data
pd.set_option("display.max_columns", None) # changing the max_columns value
data.sample(5)

,id,age,gender,height,weight,ap_hi,ap_lo,cholesterol,smoke
1959,2768,22603,1,158,66.0,120,80,3,0
19072,27236,16053,2,176,61.0,120,80,1,0
25208,35988,21418,1,157,68.0,120,80,1,0
11683,16695,18415,1,152,50.0,90,70,1,0
30534,43651,14366,1,158,65.0,140,90,1,0


### Variables

Let's take a closer look at the variables and their documentation.

__Feature | variable name | type__

* Id                       | unique ID   | continuous variable  | int
* Age                      | age         | continuous variable  | int (days)
* Gender                   | gender      | categorical variable | binary
* Height                   | height      | continuous variable  | int (cm)
* Weight                   | weight      | continuous variable  | float (kg)
* Systolic blood pressure  | ap_hi       | continuous variable  | int
* Diastolic blood pressure | ap_lo       | continuous variable  | int
* Cholesterol              | cholesterol | categorical variable | 1: normal, 2: above normal, 3: well above normal
* Smoking                  | smoke       | categorical variable | binary

## Exploratory Data Analysis (EDA)

### Plotting Objectives
Before diving into the EDA, it is good to have a clear goal in mind.

Our goal is to calculate some descriptive statistics. The following points should help explore and visualize data accordingly **given this goal**:
 * Check features and their distributions, unidimensionally.
 * Check correlation between features, bidimensionally.

### Defining Functions for EDA

In [8]:
### DATA ANALYSIS PART ###

# Checking Shape
def data_shape(data):
    print("Dataset shape: " + str(data.shape[0]) + " observations and " + str(data.shape[1]) + " features.")

# Check Size
def data_size(data):
    print("This dataset has a total of: " + str(data.size) + " entries.")

# Check Information
def data_info(data):
    print(data.name)
    print("--------------------------------------")
    data.info()
    print("--------------------------------------")  
    
# Get Unique Values - Categorical Variables
def unique_values(data):                                          # define a function (output: unique values in the categorical variables)
    for i in cols:                                                # cols is the list of features from this dataset defined in the "Importing the Dataset" section above
        if data[i].dtype == 'O':                                  # check whether features are categorical variables. 'O' stand for object type.
            print('Unique values in', i, 'are', data[i].unique()) # calls function unique() to find get unique values
            print('----------------------------------------------------------------------------------------------------')

# Check for Missing Values
def missing_values(data):
    print('Checking for missing values in the', data.name) # data.name has been defined previously in the "Importing" section
    print('------------------------------------------------------------')
    print(data.isnull().sum())
    print('------------------------------------------------------------')

# Save Data - Continuous Variables
def save_cont_data(data):
    cont_data = data.select_dtypes(include = 'number')
    return cont_data
    
# Save Data - Categorical Variable
def save_cat_data(data):
    cat_data = data.select_dtypes(exclude = 'number')
    return cat_data

# IQR Method - Detecting Outliers
def iqr_method(potential_outliers, data_copy): #arg 1 takes list of features with potential outliers, arg2 
    i = 1
    for col in potential_outliers:
        Q1 = data_copy[col].quantile(0.25)
        Q3 = data_copy[col].quantile(0.75)
        IQR = Q3 - Q1
        print(f'column {i}: {data_copy[col].name}\n------------------------')
        print('1st quantile => ',Q1)
        print('3rd quantile => ',Q3)
        print('IQR =>',IQR)

        lower_bound  = Q1-(1.5*IQR)
        print('lower_bound => ' + str(lower_bound))

        upper_bound = Q3+(1.5*IQR)
        print('upper_bound => ' + str(upper_bound))
        print("\n------------------------")
        
        i = i + 1

        data_copy[col][((data_copy[col] < lower_bound) | (data_copy[col] > upper_bound))] = np.nan  # replacing outliers with NaN


### VISUALIZATION PART ###

# Plot Probability Distributions - Continuous Variables
def pdf_plot_cont(cont_data):
    for i in cont_data:
        ax = sns.displot(cont_data[i])
        plt.show()

# Plot Bar Plots - Categorical Variables (and order by value_counts within them)
def bar_plot_cat(cat_data):
    plt.figure(figsize=(20,4))
    for i in cat_data:
        ax = sns.countplot(y = cat_data[i], order = cat_data[i].value_counts().index)
        plt.show()

# Plot Box Plots - Continuous Variables
def box_plot(potential_outliers, cont_data): # the first argument takes a list of features and the second the dataset
    for i in potential_outliers:
        ax = sns.boxplot(x = cont_data[i], orient = 'h')
        plt.show()
